# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ig" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [6]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-ig-baseline


In [6]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 4.3MB/s 


In [7]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-ig.xml.gz not found. The following files are available for downloading:

   4 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-ig.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  54 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/ig.zip

 321 MB Total size
./JW300_latest_xml_en-ig.xml.gz ... 100% of 4 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_ig.zip ... 100% of 54 MB


In [8]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-01-12 07:57:39--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.03s   

2020-01-12 07:57:39 (9.27 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-01-12 07:57:40--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-ig.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [9]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [10]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 6326/475205 lines since contained in test set.


,source_sentence,target_sentence
0,Using Ladders — Do You Make These Safety Checks ?,Iji Ubube Eme Ihe — Ị̀ Na - eme Nnyocha Ndị A ...
1,By Awake !,Site n’aka onye nta akụkọ Teta !
2,correspondent in Ireland,na Ireland


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [11]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [12]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144679 sha256=4863c3de1bedaa5001dd58d9106a4d97cde13ce9c1f9f51c8ed649178c814bde
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.14 0.00 percent complete
00:00:19.80 0.24 percent complete
00:00:38.56 0.48 percent complete
00:00:57.39 0.72 percent complete
00:01:16.48 0.96 percent complete
00:01:34.80 1.20 percent complete
00:01:53.09 1.44 percent complete
00:02:11.90 1.68 percent complete
00:02:30.37 1.92 percent complete
00:02:49.36 2.16 percent complete
00:03:08.57 2.40 percent complete
00:03:26.94 2.63 percent complete
00:03:46.23 2.87 percent complete
00:04:04.79 3.11 percent complete
00:04:23.68 3.35 percent complete
00:04:41.95 3.59 percent complete
00:05:01.28 3.83 percent complete
00:05:20.04 4.07 percent complete
00

00:16:55.79 12.69 percent complete
00:17:15.13 12.93 percent complete
00:17:35.19 13.17 percent complete
00:17:54.04 13.41 percent complete
00:18:13.27 13.65 percent complete
00:18:33.75 13.89 percent complete
00:18:53.24 14.13 percent complete
00:19:12.60 14.37 percent complete
00:19:32.36 14.61 percent complete
00:19:51.97 14.85 percent complete
00:20:11.70 15.09 percent complete
00:20:32.01 15.33 percent complete
00:20:51.44 15.57 percent complete
00:21:12.28 15.81 percent complete
00:21:32.13 16.05 percent complete
00:21:52.41 16.29 percent complete
00:22:11.89 16.53 percent complete
00:22:31.66 16.77 percent complete
00:22:51.52 17.01 percent complete
00:23:11.14 17.25 percent complete
00:23:30.33 17.49 percent complete
00:23:51.44 17.72 percent complete
00:24:11.74 17.96 percent complete
00:24:31.87 18.20 percent complete
00:24:52.43 18.44 percent complete
00:25:11.94 18.68 percent complete
00:25:31.81 18.92 percent complete
00:25:51.83 19.16 percent complete
00:26:11.69 19.40 pe

00:28:31.73 21.08 percent complete
00:28:51.20 21.32 percent complete
00:29:12.49 21.56 percent complete
00:29:32.67 21.80 percent complete
00:29:52.59 22.04 percent complete
00:30:12.49 22.28 percent complete
00:30:32.56 22.52 percent complete
00:30:52.20 22.75 percent complete
00:31:12.62 22.99 percent complete
00:31:32.41 23.23 percent complete
00:31:53.43 23.47 percent complete
00:32:13.44 23.71 percent complete


00:32:33.70 23.95 percent complete
00:32:54.36 24.19 percent complete
00:33:13.93 24.43 percent complete
00:33:32.54 24.67 percent complete
00:33:51.89 24.91 percent complete
00:34:11.30 25.15 percent complete
00:34:32.24 25.39 percent complete
00:34:52.03 25.63 percent complete
00:35:11.04 25.87 percent complete
00:35:30.22 26.11 percent complete
00:35:49.74 26.35 percent complete
00:36:09.10 26.59 percent complete
00:36:29.03 26.83 percent complete
00:36:48.44 27.07 percent complete
00:37:08.19 27.31 percent complete
00:37:27.47 27.55 percent complete
00:37:46.58 27.78 percent complete
00:38:05.66 28.02 percent complete
00:38:24.86 28.26 percent complete
00:38:43.80 28.50 percent complete
00:39:03.46 28.74 percent complete
00:39:22.21 28.98 percent complete
00:39:42.99 29.22 percent complete
00:40:02.72 29.46 percent complete
00:40:22.64 29.70 percent complete
00:40:42.11 29.94 percent complete
00:41:01.16 30.18 percent complete
00:41:20.42 30.42 percent complete
00:41:39.90 30.66 pe

00:47:06.10 34.73 percent complete
00:47:25.26 34.97 percent complete


...

01:58:19.63 88.14 percent complete
01:58:38.07 88.38 percent complete
01:58:57.10 88.62 percent complete


01:59:16.90 88.86 percent complete
01:59:37.19 89.10 percent complete
01:59:57.69 89.34 percent complete
02:00:16.97 89.58 percent complete
02:00:35.82 89.82 percent complete
02:00:54.60 90.06 percent complete
02:01:14.32 90.30 percent complete
02:01:34.50 90.54 percent complete
02:01:54.07 90.78 percent complete
02:02:13.59 91.02 percent complete
02:02:33.10 91.26 percent complete
02:02:51.51 91.50 percent complete
02:03:11.79 91.74 percent complete
02:03:30.93 91.98 percent complete
02:03:49.60 92.22 percent complete


02:04:08.83 92.46 percent complete
02:04:27.47 92.70 percent complete
02:04:46.64 92.93 percent complete
02:05:06.11 93.17 percent complete
02:05:25.67 93.41 percent complete
02:05:43.84 93.65 percent complete
02:06:02.80 93.89 percent complete
02:06:21.93 94.13 percent complete
02:06:40.96 94.37 percent complete
02:06:59.60 94.61 percent complete
02:07:18.53 94.85 percent complete
02:07:38.08 95.09 percent complete
02:07:57.28 95.33 percent complete
02:08:16.07 95.57 percent complete
02:08:34.73 95.81 percent complete
02:08:53.66 96.05 percent complete
02:09:12.34 96.29 percent complete
02:09:31.67 96.53 percent complete
02:09:50.88 96.77 percent complete
02:10:10.40 97.01 percent complete
02:10:30.20 97.25 percent complete
02:10:48.18 97.49 percent complete
02:11:06.85 97.73 percent complete
02:11:25.74 97.96 percent complete
02:11:45.60 98.20 percent complete


02:12:04.37 98.44 percent complete
02:12:23.27 98.68 percent complete
02:12:42.66 98.92 percent complete
02:13:02.28 99.16 percent complete
02:13:21.16 99.40 percent complete


02:13:40.29 99.64 percent complete
02:13:59.55 99.88 percent complete


In [13]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
“ Whatever we ask we receive from him , ” wrote the apostle John , “ because we are observing his commandments and are doing the things that are pleasing in his eyes . ”
Did Leah conspire to deceive Jacob ?
While the one under demon attack should pray fervently for relief , other true Christians may also pray in behalf of the individual who deeply desires to serve Jehovah and who is earnestly trying to resist wicked forces .
The articles on pages 5 to 9 of this issue discuss what that Kingdom is and when it will come .
It answers the following questions : What factors presently prevent diplomacy from producing peace ?
Are these mammoth sea monsters slow and clumsy in the water ?
A good counselor shows empathy , respect , and kindness
Jehovah raises up the horns of his people , causing them to be exalted , whereas he ‘ cuts down the horns of the wicked ones . ’
And it came about at the closing of the gate by dark that the men went out . ”
For instance , an angel was ask



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [2]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 2199 (delta 4), reused 5 (delta 3), pack-reused 2184
Receiving objects: 100% (2199/2199), 2.60 MiB | 1.45 MiB/s, done.
Resolving deltas: 100% (1525/1525), done.
Processing /content/joeynmt
     |████████████████████████████████| 276kB 11.5MB/s 
     |████████████████████████████████| 307kB 40.4MB/s 
     |████████████████████████████████| 51kB 9.5MB/s 
     |████████████████████████████████| 215kB 58.1MB/s 
     |████████████████████████████████| 737kB 68.5MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72136 sha256=18f9b987041cdc5803f19a9fcf70eac72f38fc6db0d7104d306349a977414a5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-id0kdhdp/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyaml

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [15]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Igbo Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman
! cp joeynmt/data/$src$tgt/vocab.txt "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.ig     test.ig	   train.en
dev.bpe.en	dev.ig	     test.en	     train.bpe.en  train.ig
dev.bpe.ig	test.bpe.en  test.en-any.en  train.bpe.ig
bpe.codes.4000	dev.en	     test.bpe.ig     test.ig	   train.en
dev.bpe.en	dev.ig	     test.en	     train.bpe.en  train.ig
dev.bpe.ig	test.bpe.en  test.en-any.en  train.bpe.ig
BPE Igbo Sentences
Gịnị mere H@@ us@@ ha@@ ị ji kwesị inwe obi ike ka o nwee ike ịkw@@ ado Chineke ?
Gịnị mere anyị ji kwesị inwe obi ike ka anyị nwee ike ịna - akwado Jehova ?
M kpere ekpere ka Jehova nye m obi ike ime ihe m kpebiri .
I@@ we ha ad@@ aj@@ ụ@@ ọla ugbu a . M na - ag@@ azi ahụ ha mgbe niile . ” — Gụọ Ilu 29 : 25 .
[ 1 ] ( par@@ ag@@ ra@@ f nke 7 ) A@@ ha a kpọrọ ụfọdụ ndị n’isiokwu a abụghị ezigbo aha ha .
Combined BPE Vocab
ạ@@
ô
ọgar@@
̇
ş
ů@@
◀
eazụ
ʺ
×


In [16]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.ig     test.ig	   train.en
dev.bpe.en	dev.ig	     test.en	     train.bpe.en  train.ig
dev.bpe.ig	test.bpe.en  test.en-any.en  train.bpe.ig  vocab.txt


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}/train.bpe"
    dev:   "{gdrive_path}/dev.bpe"
    test:  "{gdrive_path}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}/vocab.txt"
    trg_vocab: "{gdrive_path}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "{gdrive_path}/models/{name}_transformer"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

! cp joeynmt/configs/transformer_$src$tgt.yaml "$gdrive_path"

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [0]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
# !cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml
!python3 -m joeynmt train "$gdrive_path/transformer_$src$tgt.yaml"

2020-01-12 19:01:36,159 Hello! This is Joey-NMT.
2020-01-12 19:01:37,233 Total params: 12188928
2020-01-12 19:01:37,234 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [0]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

In [0]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"